In [0]:
# Retrieve the parameters passed from the downstream_audit notebook
id = dbutils.widgets.get("id")
data_team_email = dbutils.widgets.get("Data_Team_email")
data_team_password = dbutils.widgets.get("Data_Team_password")
Schema_registry_details=dbutils.widgets.get("Schema_registry_details")
print(id)

# Convert id from string to integer
id = int(id)

# Query the downstream_audit table
df = spark.sql(f"select * from ds_training_1.default.downstream_audit where id={id}")
display(df)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3477860330566271>, line 2
      1 # Retrieve the parameters passed from the downstream_audit notebook
----> 2 id = dbutils.widgets.get("id")
      3 data_team_email = dbutils.widgets.get("Data_Team_email")
      4 data_team_password = dbutils.widgets.get("Data_Team_password")

File /databricks/python_shell/dbruntime/WidgetHandlerImpl.py:77, in WidgetsHandlerImpl.get(self, name)
     37 def get(self, name):
     38     """ Returns the current value of a widget with the given name.
     39 
     40     :param name: Name of the argument to be accessed
   (...)
     75         ```
     76     """
---> 77     return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *args)
   1349 comma

In [0]:
import json
# Use the correct column name
source_table = df.select("source_table").collect()[0][0]
table_owner = df.select("table_owner").collect()[0][0]
upstream_changed_schema_info=json.loads(df.select("upstream_changed_schema_info").collect()[0][0])
TablesInfo=json.loads(df.select("TablesInfo").collect()[0][0])
PermissionInfo=json.loads(df.select("PermissionInfo").collect()[0][0])
GroupInfo=json.loads(df.select("GroupInfo").collect()[0][0])
JobInfo=json.loads(df.select("JobInfo").collect()[0][0])
NotebookInfo=json.loads(df.select("NotebookInfo").collect()[0][0])
DashboardInfo=json.loads(df.select("DashboardInfo").collect()[0][0])
QueryInfo=json.loads(df.select("QueryInfo").collect()[0][0])
PipelineInfo=json.loads(df.select("pipelineInfo").collect()[0][0])
ModelsInfo=json.loads(df.select("ModelsInfo").collect()[0][0])
FileInfo=json.loads(df.select("FileInfo").collect()[0][0])

In [0]:
import smtplib
from email.mime.multipart import MIMEMultipart
from pyspark.sql import functions as F
from email.mime.text import MIMEText
import json
from pyspark.sql import SparkSession

# SMTP server configuration
smtp_server = "smtp.gmail.com"  
smtp_port = 587  
# smtp_user = "danduprolu.stuthi@latentview.com"  
# smtp_password = "ippr iwgm aqji ddlw"  
smtp_user = data_team_email
smtp_password = data_team_password
# Function to send email notifications
def send_email(to_email, subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = smtp_user
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(smtp_user, smtp_password)
            server.send_message(msg)
        print(f"Email sent successfully to {to_email}")
    except Exception as e:
        print(f"Failed to send email to {to_email}: {e}")

# Initialize Spark session
spark = SparkSession.builder.appName("SchemaRegistryUpdate").getOrCreate()

def update_schema_registry_table(id):
    try:
        # Construct the SQL query to update the table (non-Delta version)
        update_query = (
            f"UPDATE {Schema_registry_details} "
            f"SET schema_change_alert_status = 'Notified' "
            f"WHERE id = '{id}' "
        )
        
        # Execute the update query using Spark SQL
        print(id)
        spark.sql(update_query)
        
        print(f"Updated the schema registry table {Schema_registry_details} successfully!")
    
    except Exception as e:
        print(f"Failed to update the schema registry table: {e}")

In [0]:
def notify_owner(source_table, table_owner, upstream_changes):
    # Construct the message for the owner
    body = (
        f"Dear {table_owner},\n\n"
        f"The following upstream changes have been made to the table '{source_table}':\n\n"
    )

    for change in upstream_changes:
        body += (
            f"- Schema Version: {change['schema_version']}\n"
            f"- Modified By: {change['modified_by']}\n"
            f"- Modified Timestamp: {change['modified_timestamp']}\n"
            f"- Change Type: {change['change_type']}\n"
            f"- Column Name: {change['column_name'] if change['column_name'] else 'N/A'}\n"
            f"- Schema JSON: {change['schema_json'] if change['schema_json'] else 'N/A'}\n"
            f"- Schema Name: {change['schema_name']}\n"
            f"- Created By: {change['created_by']}\n\n"
        )

    body += "Please review these changes to ensure your downstream processes remain unaffected.\n\nBest regards,\nYour Data Team"

    # Send the email to the owner
    send_email(table_owner, f"Upstream Changes Notification for {source_table}", body)

    return body  # Return the constructed message for later use

def details(source_table, table_owner, upstream_changes):
    body = (
        f"The following upstream changes have been made to the table '{source_table}':\n\n"
    )
    for change in upstream_changes:
        body += (
            f"- Schema Version: {change['schema_version']}\n"
            f"- Modified By: {change['modified_by']}\n"
            f"- Modified Timestamp: {change['modified_timestamp']}\n"
            f"- Change Type: {change['change_type']}\n"
            f"- Column Name: {change['column_name'] if change['column_name'] else 'N/A'}\n"
            f"- Schema JSON: {change['schema_json'] if change['schema_json'] else 'N/A'}\n"
            f"- Schema Name: {change['schema_name']}\n"
            f"- Created By: {change['created_by']}\n\n"
        )
    body += "Please review these changes to ensure your downstream processes remain unaffected."
    return body 

def notify_downstream_entities(
    source_table, table_owner, upstream_changed_schema_info, TablesInfo, PermissionInfo, 
    GroupInfo, JobInfo, NotebookInfo, DashboardInfo, QueryInfo, PipelineInfo, ModelsInfo, FileInfo
):
    # Dictionary to hold email notifications for unique recipients
    email_notifications = {}
    upstream_changes = []

    # Collect all upstream changes for the owner
    for upstream in upstream_changed_schema_info:
        upstream_changes.append(upstream)

    # Notify the owner with all upstream changes at once and get the message
    
    schema_changes_message=details(source_table, table_owner, upstream_changes)

    # Notify for each table in TablesInfo
    for table in TablesInfo:
        user = table[0]['created_by_email']
        table_name = table[0]['name']
        catalog_name = table[0]['catalog_name']
        schema_name = table[0]['schema_name']

        message = (
            f"Your table '{table_name}' in catalog '{catalog_name}' and schema '{schema_name}' has been affected due to recent changes.\n\n"
            f"Details of the upstream change:\n"
            f"- Table Name: {table_name}\n"
            f"- Catalog Name: {catalog_name}\n"
            f"- Schema Name: {schema_name}\n\n"
            f"Please review the changes to ensure your downstream processes remain unaffected."
        )

        if user not in email_notifications:
            email_notifications[user] = {'messages': [], 'unique_info': set()}
        email_notifications[user]['messages'].append(message)

    # Notify based on permissions in PermissionInfo
    for perm in PermissionInfo:
        principal = perm['Principal']
        object_key = perm['ObjectKey']
        action_type = perm['ActionType']
        email = perm['created_by_email_id']
        if '@' in principal:
            message = (
                f"You have permissions on the table '{object_key}' which has upstream changes.\n\n"
                f"Action Type: {action_type}\n\n"
                f"Details of the upstream change:\n"
                f"- Table Name: {object_key}\n\n"
                f"Please review the changes to ensure your access and operations remain unaffected."
            )
            if email not in email_notifications:
                email_notifications[email] = {'messages': [], 'unique_info': set()}
            email_notifications[email]['messages'].append(message)

    # Notify for each group in GroupInfo
    for group in GroupInfo:
        previleges = group['Previleges']
        ObjectKey = group['ObjectKey']
        if group['group_details']:
            group_name = group['group_details'][0]['Group Name']
            group_id = group['group_details'][0]['Group ID']
            email_ids = group['group_details'][0]['Email IDs']
        else:
            email_ids = []

        for member_email in email_ids:
            message = (
                f"Your group '{group_name}' has permissions: {previleges} on the table '{ObjectKey}' which is affected by upstream changes.\n\n"
                f"Group Details:\n"
                f"- Group ID: {group_id}\n\n"
                f"Please review these changes as they may impact the operations for your group."
 )
            if member_email not in email_notifications:
                email_notifications[member_email] = {'messages': [], 'unique_info': set()}
            email_notifications[member_email]['messages'].append(message)

    # Notify for each job in JobInfo
    for job in JobInfo:
        created_by = job[0]['created_by_email']
        entity_name = job[0]['entity_name']
        workspace_id = job[0]['workspace_id']
        job_id = job[0]['job_id']

        message = (
            f"Your job '{entity_name}' may be affected by upstream changes in the source table '{source_table}'.\n\n"
            f"Job Details:\n"
            f"- Workspace ID: {workspace_id}\n"
            f"- Job ID: {job_id}\n\n"
            f"Please review these changes to assess any impact on the job's operations."
        )
        if created_by not in email_notifications:
            email_notifications[created_by] = {'messages': [], 'unique_info': set()}
        email_notifications[created_by]['messages'].append(message)

    # Notify for each notebook in NotebookInfo
    for notebook in NotebookInfo:
        created_by = notebook[0]['created_by_email']
        workspace_id = notebook[0]['workspace_id']
        notebook_id = notebook[0]['notebook_id']

        message = (
            f"Your notebook may be affected by upstream changes in the source table '{source_table}'.\n\n"
            f"Notebook Details:\n"
            f"- Workspace ID: {workspace_id}\n"
            f"- Notebook ID: {notebook_id}\n\n"
            f"Please review these changes to assess any impact on the notebook's functionality."
        )
        if created_by not in email_notifications:
            email_notifications[created_by] = {'messages': [], 'unique_info': set()}
        email_notifications[created_by]['messages'].append(message)

    # Notify for each dashboard in DashboardInfo
    for dashboard in DashboardInfo:
        created_by = dashboard[0]['created_by_email']
        workspace_id = dashboard[0]['workspace_id']
        dashboard_id = dashboard[0]['dashboard_id']

        message = (
            f"Your dashboard may be affected by upstream changes in the source table '{source_table}'.\n\n"
            f"Dashboard Details:\n"
            f"- Workspace ID: {workspace_id}\n"
            f"- Dashboard ID: {dashboard_id}\n\n"
            f"Please review these changes to assess any impact on the dashboard's performance."
        )
        if created_by not in email_notifications:
            email_notifications[created_by] = {'messages': [], 'unique_info': set()}
        email_notifications[created_by]['messages'].append(message)

    # Notify for each query in QueryInfo
    for query in QueryInfo:
        created_by = query[0]['created_by_email']
        entity_name = query[0]['entity_name']
        workspace_id = query[0]['workspace_id']
        query_id = query[0]['query_id']

        message = (
            f"Your query '{entity_name}' may be affected by upstream changes in the source table '{source_table}'.\n\n"
            f"Query Details:\n"
            f"- Workspace ID: {workspace_id}\n"
            f"- Query ID: {query_id}\n\n"
            f"Please review these changes to assess any impact on the query's results."
        )
        if created_by not in email_notifications:
            email_notifications[created_by] = {'messages': [], 'unique_info': set()}
        email_notifications[created_by]['messages'].append(message)

    # Notify for each file in FileInfo
    for file in FileInfo:
        created_by = file[0]['created_by_email']
        workspace_id = file[0]['workspace_id']
        file_id = file[0]['file_id']

        message = (
            f"Your file may be affected by upstream changes in the source table '{source_table}'.\n\n"
            f"File Details:\n"
            f"- Workspace ID: {workspace_id}\n"
            f"- File ID: {file_id}\n\n"
            f"Please review these changes to assess any impact on the file's data."
        )
        if created_by not in email_notifications:
            email_notifications[created_by] = {'messages': [], 'unique_info': set()}
        email_notifications[created_by]['messages'].append(message)
    if table_owner not in email_notifications:
        owner_message = notify_owner(source_table, table_owner, upstream_changes)

    # Send a single email to each unique recipient
    for recipient, info in email_notifications.items():
        messages = info['messages']
        unique_info = info['unique_info']

        # Remove duplicate messages
        unique_messages = []
        for message in messages:
            if message not in unique_messages:
                unique_messages.append(message)

        # Append the upstream changes message
        body = (
            f"Dear {recipient},\n\n"
            f"You have been affected by the following upstream changes:\n\n"
            f"{''.join(unique_messages)} \n\n"
            f"{schema_changes_message} \n\n"
            f"Best regards,\nYour Data Team"
        )
        send_email(recipient, f"Upstream Changes Notification for {source_table}", body)

    print("Emails sent to downstream users!")
    for upstream in upstream_changed_schema_info:
        update_schema_registry_table(upstream['id'])

In [0]:
notify_downstream_entities(source_table, table_owner, upstream_changed_schema_info, TablesInfo, PermissionInfo, 
    GroupInfo, JobInfo, NotebookInfo, DashboardInfo, QueryInfo, PipelineInfo, ModelsInfo, FileInfo)

Email sent successfully to vishal.kokkula@latentview.com
Email sent successfully to danduprolu.stuthi@latentview.com
Email sent successfully to kudumalauday.kumarreddy@latentview.com
Email sent successfully to brindavivek.kotha@latentview.com
Failed to send email to N/A: {'N/A': (553, b'5.1.3 The recipient address <N/A> is not a valid RFC 5321 address. For more\n5.1.3 information, go to\n5.1.3  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.1.3 specifications. d75a77b69052e-461452c474fsm27299171cf.93 - gsmtp')}
Emails sent to downstream users!
31
Updated the schema registry table ds_training_1.default.schema_registry successfully!
26
Updated the schema registry table ds_training_1.default.schema_registry successfully!
27
Updated the schema registry table ds_training_1.default.schema_registry successfully!
28
Updated the schema registry table ds_training_1.default.schema_registry successfully!
